In [3]:
import utils

import numpy as np
from IPython.display import HTML, display, Markdown
import pandas as pd
import ot
# import partial_gw as pgw
import partial_gw_1 as pgw
import matplotlib.pyplot as plt

# from subset_select_ipot_non_uniform_git import subset_select_ipot as SS_ipot
# from subset_select_non_uniform_entropic_FISTA_git import subset_select_non_uniform_FISTA as SS

# from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score


c:\Users\bilal\anaconda3\envs\pytorch\lib\site-packages\ot\backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


# Same Domain

In [4]:
from utils import make_data
from utils import draw_p_u_dataset_scar
# import pgw.compute_cost_matrices as cost_matrix

datasets = ['mushrooms', 'shuttle', 'pageblocks', 'usps', 'connect-4', 'spambase', 'mnist', 'mnist-colored']
# datasets = ['mushrooms']
prior_lst = [0.518, 0.786, 0.898, 0.167, 0.658, 0.394, 0.1, 0.1]

In [5]:
#https://pythonot.github.io/gen_modules/ot.partial.html

from ot.partial import partial_wasserstein as FR_POT

n_unl = 800
n_pos = 400
nb_reps = 10
nb_dummies = 10



acc_FR_pW_avg = []
precision_FR_pW_avg = []
recall_FR_pW_avg = []
f1_scores_FR_pW_avg = []
roc_auc_FR_pW_avg = []
yu_dict_FR_pW = {}
yhat_dict_FR_pW = {}
nu_star_dict_FR_pW = {}


for ds_idx in range(0, len(datasets)):
    print(datasets[ds_idx])
    mu = np.ones(n_pos)/n_pos
    nu = np.ones(n_unl)/n_unl
    
    ds = datasets[ds_idx]
    if ds != "mnist-colored":
        ds_p = ds
        ds_ul = ds
    else:
        ds_p = 'mnist_color_change_p'
        ds_ul = 'mnist_color_change_u'
    
    acc_FR_pW = []
    precision_FR_pW = []
    recall_FR_pW = []
    f1_scores_FR_pW = []
    roc_auc_FR_pW = []
    nu_star_lst = []
    yu_list_FR_pW = []
    yhat_list_FR_pW= []
    
    for reps in range(0, nb_reps):
        
        xp_df, xu_df, yu_df = draw_p_u_dataset_scar(ds_p, ds_ul, n_pos, n_unl, prior_lst[ds_idx], reps)
        xp = xp_df.to_numpy()
        xu = xu_df.to_numpy()
        yu = yu_df.to_numpy()
        
        yu_list_FR_pW.append(yu)
        M = np.linalg.norm(np.expand_dims(xp,1) - xu, axis=-1)

        P = FR_POT(mu, nu, M, m = prior_lst[ds_idx])
        nu_star = np.sum(P,axis=0)
        nu_star_lst.append(nu_star)
        nu_star_sorted_idx = np.argsort(-nu_star)
        # nu_start_selected_idx = nu_star_sorted_idx[0:int(prior_lst[i]*len(nu_star))]
        nu_start_selected_idx = nu_star_sorted_idx[0:int(np.sum(yu))]
        yu_hat = np.zeros_like(yu)
        yu_hat[nu_start_selected_idx] = 1
        acc_FR_pW.append(np.mean(yu_hat == yu))
        yhat_list_FR_pW.append(yu_hat)
        precision_FR_pW_rep, recall_FR_pW_rep, f1_score_FR_pW_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')

        
        precision_FR_pW.append(precision_FR_pW_rep)
        recall_FR_pW.append(recall_FR_pW_rep)
        f1_scores_FR_pW.append(f1_score_FR_pW_rep)
        # roc_auc_FR_pW.append(roc_auc_score(yu, yu_hat))
        roc_auc_FR_pW.append(roc_auc_score(yu, nu_star))
        
    yu_dict_FR_pW[ds_idx] = yu_list_FR_pW
    yhat_dict_FR_pW[ds_idx] = yhat_list_FR_pW
        
    precision_FR_pW_avg.append(np.mean(precision_FR_pW))
    recall_FR_pW_avg.append(np.mean(recall_FR_pW))
    f1_scores_FR_pW_avg.append(np.mean(f1_scores_FR_pW))
    acc_FR_pW_avg.append(np.mean(acc_FR_pW))
    roc_auc_FR_pW_avg.append(np.mean(roc_auc_FR_pW))
    
    print("precision FR_pW= ", precision_FR_pW_avg[ds_idx])
    print("recall FR_pW= ", recall_FR_pW_avg[ds_idx])
    print("f1-score FR_pW= ", f1_scores_FR_pW_avg[ds_idx])
    print("roc auc FR_pW=", roc_auc_FR_pW_avg[ds_idx])
    print("acc_FR_pW_avg=", acc_FR_pW_avg[ds_idx])

mushrooms
precision FR_pW=  0.9946859903381642
recall FR_pW=  0.9946859903381642
f1-score FR_pW=  0.9946859903381642
roc auc FR_pW= 0.9948486896448149
acc_FR_pW_avg= 0.9945
shuttle
precision FR_pW=  0.9767515923566877
recall FR_pW=  0.9767515923566877
f1-score FR_pW=  0.9767515923566877
roc auc FR_pW= 0.9466828988298029
acc_FR_pW_avg= 0.9635
pageblocks
precision FR_pW=  0.9547353760445685
recall FR_pW=  0.9547353760445685
f1-score FR_pW=  0.9547353760445685
roc auc FR_pW= 0.7817107140430737
acc_FR_pW_avg= 0.91875
usps
precision FR_pW=  0.9120300751879699
recall FR_pW=  0.9120300751879699
f1-score FR_pW=  0.9120300751879699
roc auc FR_pW= 0.9475544182795821
acc_FR_pW_avg= 0.97075
connect-4
precision FR_pW=  0.703041825095057
recall FR_pW=  0.703041825095057
f1-score FR_pW=  0.703041825095057
roc auc FR_pW= 0.5665784324609364
acc_FR_pW_avg= 0.6094999999999999
spambase
precision FR_pW=  0.6073015873015872
recall FR_pW=  0.6073015873015872
f1-score FR_pW=  0.6073015873015872
roc auc FR_pW=

In [6]:
results_UCI = {'dataset': datasets,
               '$\pi$': prior_lst,
               'acc FR-PW': acc_FR_pW_avg,
               'roc-auc FR-PW': roc_auc_FR_pW_avg,
               }
results_UCI = pd.DataFrame(data=results_UCI)
results_UCI

,dataset,$\pi$,acc FR-PW,roc-auc FR-PW
0,mushrooms,0.518,0.99450,0.994849
1,shuttle,0.786,0.96350,0.946683
2,pageblocks,0.898,0.91875,0.781711
3,usps,0.167,0.97075,0.947554
4,connect-4,0.658,0.60950,0.566578
5,spambase,0.394,0.69075,0.677026
6,mnist,0.100,0.99150,0.976845
7,mnist-colored,0.100,0.97675,0.935961


# Across Domains

In [7]:
n_unl = 100
n_pos = 100
nb_reps = 10
nb_dummies = 10

acc_FR_pW_avg = []
precision_FR_pW_avg = []
recall_FR_pW_avg = []
f1_scores_FR_pW_avg = []
roc_auc_FR_pW_avg = []
yu_dict_FR_pW = {}
yhat_dict_FR_pW = {}
nu_star_dict_FR_pW = {}


acc_FR_pW_avg = []
precision_FR_pW_avg = []
recall_FR_pW_avg = []
f1_scores_FR_pW_avg = []
roc_auc_FR_pW_avg = []
yu_dict_FR_pW = {}
yhat_dict_FR_pW = {}
nu_star_dict_FR_pW = {}


domain_u = ['surf_Caltech10', 'surf_amazon', 'surf_webcam', 'surf_dslr']

for ds_idx in range(0, len(domain_u)):
    ds_ul = domain_u[ds_idx]
    print('labeled='+'surf_Caltech10')
    print('unlabeled='+ ds_ul)
    
    mu = np.ones(n_pos)/n_pos
    nu = np.ones(n_unl)/n_unl
    
    acc_FR_pW = []
    precision_FR_pW = []
    recall_FR_pW = []
    f1_scores_FR_pW = []
    roc_auc_FR_pW = []
    nu_star_lst = []
    yu_list_FR_pW = []
    yhat_list_FR_pW= []
    
    for reps in range(0, nb_reps):
        xp_df, xu_df, yu_df = draw_p_u_dataset_scar('surf_Caltech10', ds_ul, n_pos, n_unl, 0.1, reps)
        xp = xp_df.to_numpy()
        xu = xu_df.to_numpy()
        yu = yu_df.to_numpy()
        
        yu_list_FR_pW.append(yu)
        M = np.linalg.norm(np.expand_dims(xp,1) - xu, axis=-1)

        P = FR_POT(mu, nu, M, m = prior_lst[ds_idx])
        nu_star = np.sum(P,axis=0)
        nu_star_lst.append(nu_star)
        nu_star_sorted_idx = np.argsort(-nu_star)
        # nu_start_selected_idx = nu_star_sorted_idx[0:int(prior_lst[i]*len(nu_star))]
        nu_start_selected_idx = nu_star_sorted_idx[0:int(np.sum(yu))]
        yu_hat = np.zeros_like(yu)
        yu_hat[nu_start_selected_idx] = 1
        acc_FR_pW.append(np.mean(yu_hat == yu))
        yhat_list_FR_pW.append(yu_hat)
        precision_FR_pW_rep, recall_FR_pW_rep, f1_score_FR_pW_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')

        
        precision_FR_pW.append(precision_FR_pW_rep)
        recall_FR_pW.append(recall_FR_pW_rep)
        f1_scores_FR_pW.append(f1_score_FR_pW_rep)
        # roc_auc_FR_pW.append(roc_auc_score(yu, yu_hat))
        roc_auc_FR_pW.append(roc_auc_score(yu, nu_star))
        
    yu_dict_FR_pW[ds_idx] = yu_list_FR_pW
    yhat_dict_FR_pW[ds_idx] = yhat_list_FR_pW
        
    precision_FR_pW_avg.append(np.mean(precision_FR_pW))
    recall_FR_pW_avg.append(np.mean(recall_FR_pW))
    f1_scores_FR_pW_avg.append(np.mean(f1_scores_FR_pW))
    acc_FR_pW_avg.append(np.mean(acc_FR_pW))
    roc_auc_FR_pW_avg.append(np.mean(roc_auc_FR_pW))
    
    print("precision FR_pW= ", precision_FR_pW_avg[ds_idx])
    print("recall FR_pW= ", recall_FR_pW_avg[ds_idx])
    print("f1-score FR_pW= ", f1_scores_FR_pW_avg[ds_idx])
    print("roc auc FR_pW=", roc_auc_FR_pW_avg[ds_idx])
    print("acc_FR_pW_avg=", acc_FR_pW_avg[ds_idx])

domain_u = ['decaf_caltech', 'decaf_amazon', 'decaf_webcam', 'decaf_dslr']

for ds_idx in range(0, len(domain_u)):
    ds_ul = domain_u[ds_idx]
    print('labeled='+'decaf_Caltech10')
    print('unlabeled=' + ds_ul)
    mu = np.ones(n_pos)/n_pos
    nu = np.ones(n_unl)/n_unl
    
    acc_FR_pW = []
    precision_FR_pW = []
    recall_FR_pW = []
    f1_scores_FR_pW = []
    roc_auc_FR_pW = []
    nu_star_lst = []
    yu_list_FR_pW = []
    yhat_list_FR_pW= []
    
    for reps in range(0, nb_reps):
        
        xp_df, xu_df, yu_df = draw_p_u_dataset_scar('decaf_caltech', ds_ul, n_pos, n_unl, 0.1, reps)
        xp = xp_df.to_numpy()
        xu = xu_df.to_numpy()
        yu = yu_df.to_numpy()
        
        yu_list_FR_pW.append(yu)
        M = np.linalg.norm(np.expand_dims(xp,1) - xu, axis=-1)

        P = FR_POT(mu, nu, M, m = prior_lst[ds_idx])
        nu_star = np.sum(P,axis=0)
        nu_star_lst.append(nu_star)
        nu_star_sorted_idx = np.argsort(-nu_star)
        # nu_start_selected_idx = nu_star_sorted_idx[0:int(prior_lst[i]*len(nu_star))]
        nu_start_selected_idx = nu_star_sorted_idx[0:int(np.sum(yu))]
        yu_hat = np.zeros_like(yu)
        yu_hat[nu_start_selected_idx] = 1
        acc_FR_pW.append(np.mean(yu_hat == yu))
        yhat_list_FR_pW.append(yu_hat)
        precision_FR_pW_rep, recall_FR_pW_rep, f1_score_FR_pW_rep,  _ = precision_recall_fscore_support(yu, yu_hat, pos_label=1, average='binary')

        
        precision_FR_pW.append(precision_FR_pW_rep)
        recall_FR_pW.append(recall_FR_pW_rep)
        f1_scores_FR_pW.append(f1_score_FR_pW_rep)
        # roc_auc_FR_pW.append(roc_auc_score(yu, yu_hat))
        roc_auc_FR_pW.append(roc_auc_score(yu, nu_star))
        
    yu_dict_FR_pW[ds_idx] = yu_list_FR_pW
    yhat_dict_FR_pW[ds_idx] = yhat_list_FR_pW
        
    precision_FR_pW_avg.append(np.mean(precision_FR_pW))
    recall_FR_pW_avg.append(np.mean(recall_FR_pW))
    f1_scores_FR_pW_avg.append(np.mean(f1_scores_FR_pW))
    acc_FR_pW_avg.append(np.mean(acc_FR_pW))
    roc_auc_FR_pW_avg.append(np.mean(roc_auc_FR_pW))
    
    print("precision FR_pW= ", precision_FR_pW_avg[ds_idx])
    print("recall FR_pW= ", recall_FR_pW_avg[ds_idx])
    print("f1-score FR_pW= ", f1_scores_FR_pW_avg[ds_idx])
    print("roc auc FR_pW=", roc_auc_FR_pW_avg[ds_idx])
    print("acc_FR_pW_avg=", acc_FR_pW_avg[ds_idx])

labeled=surf_Caltech10
unlabeled=surf_Caltech10
precision FR_pW=  0.1
recall FR_pW=  0.1
f1-score FR_pW=  0.10000000000000002
roc auc FR_pW= 0.7468888888888889
acc_FR_pW_avg= 0.82
labeled=surf_Caltech10
unlabeled=surf_amazon
precision FR_pW=  0.06999999999999999
recall FR_pW=  0.06999999999999999
f1-score FR_pW=  0.07
roc auc FR_pW= 0.5336666666666666
acc_FR_pW_avg= 0.8140000000000001
labeled=surf_Caltech10
unlabeled=surf_webcam
precision FR_pW=  0.06
recall FR_pW=  0.06
f1-score FR_pW=  0.06000000000000001
roc auc FR_pW= 0.4451111111111111
acc_FR_pW_avg= 0.812
labeled=surf_Caltech10
unlabeled=surf_dslr
precision FR_pW=  0.0
recall FR_pW=  0.0
f1-score FR_pW=  0.0
roc auc FR_pW= 0.40555555555555556
acc_FR_pW_avg= 0.8
labeled=decaf_Caltech10
unlabeled=decaf_caltech
precision FR_pW=  0.1
recall FR_pW=  0.1
f1-score FR_pW=  0.10000000000000002
roc auc FR_pW= 0.7468888888888889
acc_FR_pW_avg= 0.82
labeled=decaf_Caltech10
unlabeled=decaf_amazon
precision FR_pW=  0.06999999999999999
recall F

In [8]:
results_UCI = {'dataset': ['surf C -> surf C', 'surf C -> surf A', 'surf C -> surf W', 'surf C -> surf D', 'decaf C -> decaf C', 'decaf C -> decaf A', 'decaf C -> decaf W', 'decaf C -> decaf D'],
               '$\pi$': [0.1]*8,
               'acc FR-PW': acc_FR_pW_avg,
               'roc-auc FR-PW': roc_auc_FR_pW_avg
               }
results_UCI = pd.DataFrame(data=results_UCI)
results_UCI

,dataset,$\pi$,acc FR-PW,roc-auc FR-PW
0,surf C -> surf C,0.1,0.820,0.746889
1,surf C -> surf A,0.1,0.814,0.533667
2,surf C -> surf W,0.1,0.812,0.445111
3,surf C -> surf D,0.1,0.800,0.405556
4,decaf C -> decaf C,0.1,0.820,0.766611
5,decaf C -> decaf A,0.1,0.818,0.534889
6,decaf C -> decaf W,0.1,0.820,0.561111
7,decaf C -> decaf D,0.1,0.804,0.461667
